## This file has the code for generating the edge_datacenters.xml setting file for the PureEdgeSim simulator

In [1]:
# Requirements
import pickle
from lxml import etree

In [2]:
# AP coordinates
with open('AP_locations_1100m_45m.pickle', 'rb') as f:
    AP_coords = pickle.load(f)

# High capacity edge server coordinates
with open('Server_locations_20_1100m_45m.pickle', 'rb') as f:
    s_coords_20 = pickle.load(f)

# Low capacity edge server coordinates
with open('Server_locations_100_1100m_45m.pickle', 'rb') as f:
    s_coords_100 = pickle.load(f)

# Topology links
with open('AP_links_1100m_45m.pickle', 'rb') as f:
    connections = pickle.load(f)

In [3]:
def low_capacity_server_specification_generator():
    '''
    Creates a generator for the low capacity server specification.
    '''
    yield 'True' # Periphery; whether edge devices can connect to an edge data center directly (via a single hop)
    yield '45' # idleConsumption in Watts (when CPU idle)
    yield '95' # maxConsumption in Watts (when CPU at 100%)
    yield 'False' # isOrchestrator
    yield '8' # cores
    yield '30000' # MIPS per core
    yield '16000' # RAM in MB
    yield '256000' # Storage in MB
    
def high_capacity_server_specification_generator():
    '''
    Creates a generator for the high capacity server specification.
    '''
    yield 'True' # Periphery; whether edge devices can connect to an edge data center directly (via a single hop)
    yield '60' # idleConsumption in Watts (when CPU idle)
    yield '110' # maxConsumption in Watts (when CPU at 100%)
    yield 'False' # isOrchestrator
    yield '16' # cores
    yield '40000' # MIPS per core
    yield '128000' # RAM in MB
    yield '2000000' # Storage in MB
    
def ap_specification_generator():
    '''
    Creates a generator for the Access Point specification.
    '''
    yield 'True' # Periphery; whether edge devices can connect to an edge data center directly (via a single hop)
    yield '0' # idleConsumption in Watts (when CPU idle)
    yield '0' # maxConsumption in Watts (when CPU at 100%)
    yield 'False' # isOrchestrator
    yield '0' # cores
    yield '0' # MIPS per core
    yield '0' # RAM in MB
    yield '0' # Storage in MB
    
def generate_file(AP_locs, server_locs, connections, server_specs_generator, AP_specs_generator):
    '''
    Generates an edge datacenter XML file.
    
    Args:
        AP_locs (list):  AP locations as a list of (x,y) tuples.
        server_locs (list): Edge server locations as a list of (x,y) tuples.
        connections (list): A list of all the links between APs, one link is defined as [start_location, end_location].
        server_specs_generator (function): Creates a generator for fetching server specification.
        AP_specs_generator (function): Creates a generator for fetching AP specification.
        
    Returns:
        bytes: An encoded string representation of the XML tree.
    '''
    doc = etree.Element('edge_datacenters') # Root of the XMl document tree
        
    only_APs = [ap for ap in AP_locs if ap not in server_locs] # As servers are co-located with APs, filtering out AP locations that are also server locations
    dc_names = create_datacenter_names(only_APs, server_locs) # Unique names for the servers and APs
        
    add_server_specification(doc, server_locs, dc_names, server_specs_generator) # Add servers to the XML tree
    add_AP_specification(doc, only_APs, dc_names, AP_specs_generator) # Add APs to the XML tree
    add_links(doc, connections, dc_names)  # Add links to the XML tree
        
    etree.indent(doc, space="    ")
    return etree.tostring(doc, encoding='UTF-8', xml_declaration=True, pretty_print=True)
        
def create_datacenter_names(AP_locs, server_locs):
    '''
    Creates unique names for the servers and APs.
    
    Args:
        AP_locs (list):  AP locations as a list of (x,y) tuples.
        server_locs (list): Edge server locations as a list of (x,y) tuples.
        
    Returns:
        dict: A dictionary where keys are server/AP locations and values are their corresponding names.
    '''
    dc_names = {}
        
    for i, loc in enumerate(server_locs):
        dc_names[loc] = "dc{}".format(i+1)
            
    for i, loc in enumerate(AP_locs):
        dc_names[loc] = "ap{}".format(i+1)
            
    return dc_names
    
def add_server_specification(doc, server_locs, dc_names, server_specs_generator):
    '''
    Adds servers with their specification to the XML tree.
    
    Args:
        doc (lxml.etree._Element): Root of the XML tree.
        server_locs (list): Edge server locations as a list of (x,y) tuples.
        dc_names (dict): A dictionary of the unique server/AP names, key is the location, value is the name.
        server_specs_generator (function): Creates a generator for fetching server specification.
        
    Returns:
        Nothing.
    '''
    for s_loc in server_locs:
        gen = server_specs_generator() # Create the generator for fetching server specification.
        s = etree.SubElement(doc, 'datacenter', name=dc_names[s_loc])
        etree.SubElement(s, 'periphery').text = next(gen)
        etree.SubElement(s, 'idleConsumption').text = next(gen)
        etree.SubElement(s, 'maxConsumption').text = next(gen)
        etree.SubElement(s, 'isOrchestrator').text = next(gen)
            
        l = etree.SubElement(s, 'location')
        etree.SubElement(l, 'x_pos').text = str(s_loc[0])
        etree.SubElement(l, 'y_pos').text = str(s_loc[1])
            
        etree.SubElement(s, 'cores').text = next(gen)
        etree.SubElement(s, 'mips').text = next(gen)
        etree.SubElement(s, 'ram').text = next(gen)
        etree.SubElement(s, 'storage').text = next(gen)
            
def add_AP_specification(doc, AP_locs, dc_names, AP_specs_generator):
    '''
    Adds APs with their specification to the XML tree.
    
    Args:
        doc (lxml.etree._Element): Root of the XML tree.
        AP_locs (list): AP locations as a list of (x,y) tuples.
        dc_names (dict): A dictionary of the unique server/AP names, key is the location, value is the name.
        AP_specs_generator (function): Creates a generator for fetching AP specification.
        
    Returns:
        Nothing.
    '''
    for ap_loc in AP_locs:
        gen = AP_specs_generator() # Create the generator for fetching AP specification.
        ap = etree.SubElement(doc, 'datacenter', name=dc_names[ap_loc])
        etree.SubElement(ap, 'periphery').text = next(gen)
        etree.SubElement(ap, 'idleConsumption').text = next(gen)
        etree.SubElement(ap, 'maxConsumption').text = next(gen)
        etree.SubElement(ap, 'isOrchestrator').text = next(gen)
            
        l = etree.SubElement(ap, 'location')
        etree.SubElement(l, 'x_pos').text = str(ap_loc[0])
        etree.SubElement(l, 'y_pos').text = str(ap_loc[1])
            
        etree.SubElement(ap, 'cores').text = next(gen)
        etree.SubElement(ap, 'mips').text = next(gen)
        etree.SubElement(ap, 'ram').text = next(gen)
        etree.SubElement(ap, 'storage').text = next(gen)
            
def add_links(doc, connections, dc_names):
    '''
    Adds topology links to the XML tree.
    
    Args:
        doc (lxml.etree._Element): Root of the XML tree.
        connections (list): A list of all the links between APs, one link is defined as [start_location, end_location].
        dc_names (dict): A dictionary of the unique server/AP names, key is the location, value is the name.
        
    Returns:
        Nothing.
    '''
    n = etree.SubElement(doc, 'network_links')
    for start_point, end_point in connections:
        link = etree.SubElement(n, 'link')
        etree.SubElement(link, 'from').text = dc_names[start_point]
        etree.SubElement(link, 'to').text = dc_names[end_point]

In [4]:
low_cap_dc_file = generate_file(AP_coords, s_coords_100, connections, low_capacity_server_specification_generator, ap_specification_generator)
high_cap_dc_file = generate_file(AP_coords, s_coords_20, connections, high_capacity_server_specification_generator, ap_specification_generator)

with open('edge_datacenters_low_cap.xml', 'wb') as f:
    f.write(low_cap_dc_file)
    
with open('edge_datacenters_high_cap.xml', 'wb') as f:
    f.write(high_cap_dc_file)